# guide_bot

One of the key goals with the guide_bot software is to scale from small problems to large problems without much difference in the user interface. In this talk we will see how additional computing resources easily can be put to use.

# Live demo of advanced features

In [ ]:
import guide_bot as gb

## Describing the target
Providing a list will perform independent optimization for all values in the list. Multiple list are considered independent dimensions, so two scanned parameters of N and M points would result in N*M simulations.

In [ ]:
target = gb.Target(width=0.01, height=0.01,
                   div_horizontal=[0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8],
                   div_vertical  =[0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8],
                   min_wavelength=1.3, max_wavelength=2.5,
                   instrument_length=160, target_guide_distance=0.5)

### Locking parameters

In [ ]:
target.lock_parameters("div_horizontal", "div_vertical")

![alt](figures/powerpoint_figures/Slide07.png)

## Describing the source
The parameters for the source can also be scanned.

In [ ]:
moderator = gb.Moderator(name="fom_moderator", width=0.07, height=[0.01, 0.03, 0.05, 0.07], guide_start=2.0)


Like with the target it is possible to lock these parameters too.

### Describing additional sources
It is possible to define additional sources which the optimized guide will be analyzed on.

In [ ]:
ESS_moderator = gb.ESS_Butterfly(name="ESS_Butterfly", width=0.06, height=0.03, guide_start=2.0)

### Including analysis moderators in project
The analysis_moderator keyword argument is used to add these extra sources to the project, if there are several add them as a list.

In [ ]:
settings = {"maxiter": 2, "swarmsize": 2, "minstep": 2E-4,
            "ncount": 1E6, "ncount_analysis": 1E6, "logfile": True}

project = gb.Project(name="advanced_project_test", target=target, moderator=moderator,
                     settings=settings, analysis_moderators=ESS_moderator)

## Line of sight
![alt](figures/powerpoint_figures/Slide08.png)

### Controlling line of sight
Line of sight through a guide can be limited with los_sections that tell guide_bot to eliminate line of sight. This can be done with distance from source, here line of sight is broken between 2 m from source and 120 m from source. A line of sight section needs a line of sight breaker, currently curved guide or kink.

In [ ]:
guide = project.new_guide(name="CSCE_simple_los")
guide += gb.Straight(name="Straight", m=3)
guide += gb.Curved(name="Curved", m=4, bend="right", length=[3,12])
guide += gb.Elliptic(name="Transport", m=4)

guide.add_los_section(2, 120)
guide.print_los_sections()

### ElementPoint
It is also possible to specify points within elements using the ElementPoint class and one of the following options.
- from_start [m]
- from_end [m]
- fraction [1]

In [ ]:
guide = project.new_guide(name="CSCE_complex_los")
guide += gb.Straight(name="Straight", m=3)
guide += gb.Curved(name="Curved", m=4, bend="right", length=[3,12])
guide += gb.Elliptic(name="Transport", m=4)

guide.add_los_section(gb.ElementPoint("Straight", from_start=2.0),
                      gb.ElementPoint("Transport", from_end=20))

guide.print_los_sections()

### Technical limit: no overlapping los sections
Multiple sections that do not overlap are okay!

In [ ]:
guide = project.new_guide(name="SCCE")

guide += gb.Straight(name="Extraction", m=3)
guide += gb.Curved(name="Curved1", m=4, bend="down", length=[3,12], start_point=[4, 8])
guide += gb.Curved(name="Curved2", m=4, bend="up", length=[3,12])
guide += gb.Elliptic(name="Transport", m=4)

# Los section
guide.add_los_section(gb.ElementPoint("Extraction", from_start=0.0), gb.ElementPoint("Curved1", from_end=0.0))
guide.add_los_section(gb.ElementPoint("Curved2", from_start=0.0), gb.ElementPoint("Transport", from_end=0.0))

### Can link angles to continue straight

In [ ]:
guide = project.new_guide(name="SCCE")

guide += gb.Straight(name="Extraction", m=3)
guide += gb.Curved(name="Curved1", m=4, bend="down", length=[3,12], start_point=[4, 8])
guide += gb.Curved(name="Curved2", m=4, bend="up", length=[3,12])
guide += gb.Elliptic(name="Transport", m=4)

# Los section
guide.add_los_section(gb.ElementPoint("Extraction", from_start=0.0), gb.ElementPoint("Curved1", from_end=0.0))
guide.add_los_section(gb.ElementPoint("Curved2", from_start=0.0), gb.ElementPoint("Transport", from_end=0.0))

# Link the two los breakers
curved1 = guide.get_element("Curved1")
curved2 = guide.get_element("Curved2")
curved2.angular_diversion = curved1.angular_diversion

## Little Python lession: lambda  keyword

In [ ]:
def add_two(x, y):
    return x + y

add_two(10, 2)

In [ ]:
add_two_copy = add_two
add_two_copy(2, 3)

In [ ]:
add_two_lambda = lambda x, y : x + y
add_two_lambda(3, 10)

## User defined parameters
The parameters given to each parameter in a guide module can be a number, an interval with list syntax or a user defined parameter. The user defined parameters provide the most control!

- FixedInstrumentParameter
- RelativeFreeInstrumentParameter
- DependentInstrumentParameter

### Fixed parameters
Fixed parameters are simple, they have a name and a value.

In [ ]:
small_axis = gb.FixedInstrumentParameter("small_axis", 0.1)
pi = gb.FixedInstrumentParameter("pi", 3.14159)

### Free parameters
The free parameters have a static interval given at the start, but can have dynamic limits and illegal intervals.

In [ ]:
guide_width = gb.RelativeFreeInstrumentParameter("guide_width", 0.03, 0.10)
guide_height = gb.RelativeFreeInstrumentParameter("guide_width", 0.03, 0.15)

# guide height will now be larger than guide_width!
guide_height.add_lower_dynamic(guide_width, lambda x : x)

# guide height will now be pi larger than guide_width!
guide_height.add_lower_dynamic(guide_width, lambda x, a : a*x, constants=[pi]) 

![alt](figures/powerpoint_figures/Slide09.png)

![alt](figures/powerpoint_figures/Slide10.png)

### Dependent parameters
These can be calculated from one or more other parameters, these can be free, fixed or other dependents.

In [ ]:
guide_width = gb.RelativeFreeInstrumentParameter("guide_width", 0.03, 0.15)
guide_height = gb.DependentInstrumentParameter("guide_height", guide_width, lambda x : 1.2*x)

### Usage example

In [ ]:
guide = project.new_guide(name="Balistic_straight")

guide_width = gb.RelativeFreeInstrumentParameter("guide_width", 0.03, 0.15)
guide_height = gb.RelativeFreeInstrumentParameter("guide_height", 0.04, 0.15)

guide += gb.Elliptic(name="defocusing", m=4,
                     end_width=guide_width, end_height=guide_height,
                     minor_axis_x=guide_width, minor_axis_y=guide_height)
guide += gb.Straight(name="transport", m=3, start_width=guide_width, start_height=guide_height,
                     end_width=guide_width, end_height=guide_height, length=[70, 140])
guide += gb.Elliptic(name="focusing", m=4, start_width=guide_width, start_height=guide_height,
                     minor_axis_x=guide_width, minor_axis_y=guide_height)

![alt](figures/controlled_guide.png)

## Running with cluster
Big projects should not be executed on a laptop, but on a high performance computer. With guide_bot one needs a configuration file for the cluster that has guide_bot and McStas installed. In those cases McStas is not necessary on the local machine! 

In [ ]:
from guide_bot.cluster import SLURM
DMSC = SLURM.ClusterSLURM(cluster_name="DMSC")

project.write(cluster=DMSC)

Write the project to disk, send it to a cluster and run the launch_all command, then every guide is optimized in parallel.

## Visualization
In addition to the plot_guide and plot_any_monitor widgets there are 4 additional widgets:
- compare_monitors
- compare_monitors_scan
- compare_moderators
- plot_sum

Demo in different notebook!

## Conclusion
guide_bot is still under development, but is already a powerful tool for guide design. The crucial detail is that the user interface keeps simple tasks simple, yet scales to more advanced tasks in an elegant way.

Provides the tools to design thousands of guides and to find the best option among them.